<a href="https://colab.research.google.com/github/mahalingamagesthian/learningai/blob/main/5_Comparing_DiffusionModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model is inspired from https://www.marktechpost.com/2025/05/05/a-coding-guide-to-compare-three-stability-ai-diffusion-models-v1-5-v2-base-sd3-medium-diffusion-capabilities-side-by-side-in-google-colab-using-gradio/ Whilst this run from Google Colab, there are some restrictions on the GPU Usage. Hence the comparison is limited to two models.

In [ ]:
from huggingface_hub import notebook_login

# This will pop up for your token from huggingface
notebook_login()

In [ ]:
# Install gradio
!pip install gradio

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusion3Pipeline
import gradio as gr


device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

In [ ]:
pipe1 = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None
).to(device)
pipe1.enable_attention_slicing()

In [ ]:
pipe2 = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-base",
    torch_dtype=torch.float16,
    safety_checker=None
).to(device)
pipe2.enable_attention_slicing()

In [ ]:
#Function to generate the image based on the prompt
def generate(prompt, steps, scale):
    img1 = pipe1(prompt, num_inference_steps=steps, guidance_scale=scale).images[0]
    img2 = pipe2(prompt, num_inference_steps=steps, guidance_scale=scale).images[0]
    return img1, img2

In [ ]:
def choose(selection):
    return f"✅ You selected: **{selection}**"

#This will be a Online Form, Clicking on "Generate Images" will create the images
#in the boxes.
# Prompt can be changed to recreate more pictures.

with gr.Blocks() as demo:
    gr.Markdown("## AI Social-Post Generator with 2 Models")
    with gr.Row():
        prompt = gr.Textbox(label="Prompt", placeholder="a fantasy landscape with castles and dragons")
        steps  = gr.Slider( 1, 100, value=50, step=1,     label="Inference Steps")
        scale  = gr.Slider( 1.0, 20.0, value=7.5, step=0.1, label="Guidance Scale")
    btn = gr.Button("Generate Images")
    with gr.Row():
        out1 = gr.Image(label="Model 1: SD v1.5")
        out2 = gr.Image(label="Model 2: SD v2-base")
    sel = gr.Radio(
        ["Model 1: SD v1.5","Model 2: SD v2-base"],
        label="Select your favorite"
    )
    txt = gr.Markdown()


    btn.click(fn=generate, inputs=[prompt, steps, scale], outputs=[out1, out2])
    sel.change(fn=choose, inputs=sel, outputs=txt)


demo.launch(share=True)

**TroubleShooting**
*Enable Access to hugging face model*
Cannot access gated repo for url https://huggingface.co/stabilityai/stable-diffusion-3-medium-diffusers/resolve/main/model_index.json.
Access to model stabilityai/stable-diffusion-3-medium-diffusers is restricted and you are not in the authorized list. Visit https://huggingface.co/stabilityai/stable-diffusion-3-medium-diffusers to ask for access.

*CUDA Out of Memory* OutOfMemoryError: CUDA out of memory. Tried to allocate 160.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 130.12 MiB is free. Process 2412 has 14.61 GiB memory in use. Of the allocated memory 14.39 GiB is allocated by PyTorch, and 125.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)